In [98]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Import data


Data source: https://github.com/nirupamaprv/Analyze-AB-test-Results

In [99]:
df = pd.read_csv('./data/ab_edited.csv')
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [100]:
df.groupby('group').describe()

converted                                                user_id  \
              count      mean       std  min  25%  50%  75%  max     count   
group                                                                        
control    145274.0  0.120386  0.325414  0.0  0.0  0.0  0.0  1.0  145274.0   
treatment  145311.0  0.118807  0.323563  0.0  0.0  0.0  0.0  1.0  145311.0   

                                                                      \
                    mean           std       min       25%       50%   
group                                                                  
control    788164.072594  91287.914601  630002.0  709279.5  788128.5   
treatment  787845.618446  91161.258854  630000.0  708746.5  787874.0   

                                
                 75%       max  
group                           
control    867208.25  945998.0  
treatment  866718.50  945999.0

In [101]:
old = df.query("group == 'control'")['converted']
new = df.query("group == 'treatment'")['converted']

# A/B Test (using `stats` package)

In [102]:
sm.stats.proportions_ztest([convert_old, convert_new], [n_old, n_new], alternative='smaller')

(1.3116075339133115, 0.905173705140591)

In [112]:
convert_old = sum(old)
convert_new = sum(new)
n_old = len(df.query("group == 'control'"))
n_new = len(df.query("group == 'treatment'"))
z_score, p_value = sm.stats.proportions_ztest([convert_old, convert_new], [n_old, n_new], alternative='smaller')
z_score, p_value

(1.3116075339133115, 0.905173705140591)

# A/B Test (using `numpy`)

**Assumptions**:
- X and Y are independent
- X and Y have same variance $\sigma^2$
- X and Y from normal distribution
---

- For 2nd assumption: Perform Levene test 
    * Null hypothesis: samples have same variances

In [119]:
from scipy.stats import levene
levene(old, new)

LeveneResult(statistic=1.7203126672199422, pvalue=0.18965383907560016)

**Conclusion**: X and Y have same population variance

- Calculate $\bar{X}$ and $\bar{Y}$

In [113]:
mean_old = np.mean(old) 
mean_new = np.mean(new) 
mean_old, mean_new

(0.1203863045004612, 0.11880724790277405)

- Calculate $S_x$ and $S_y$

In [114]:
s_old = np.std(old, ddof = 1) 
s_new = np.std(new, ddof = 1) 
s_old, s_new

(0.32541384592046235, 0.32356267742508843)

- Calculate $N_x$ and $N_y$

In [115]:
n_old = len(old) 
n_new = len(new) 
n_old, n_new

(145274, 145311)

If $N_x <> N_y$:

Pooled Standard Deviation: $S_p = \sqrt{\frac{(N_x-1)S_x^2+(N_y-1)S_y^2}{N_x+N_y-2}}$

Std of mean: $S_{\bar{X} - \bar{Y}} = S_p\sqrt{1/N_x+1/N_y} $

Test Statistics: $T = \frac{\bar{X} - \bar{Y}}{S_{\bar{X} - \bar{Y}}}$


Distribution: $ T$ ~ $t(N_x+N_y-2) $

---
If $N_x = N_y$:

Pooled Standard Deviation: $S_p = \sqrt{S_x^2+S_y^2}$

Std of mean: $S_{\bar{X} - \bar{Y}} = S_p\sqrt{2/N} $

Test Statistics: $T = \frac{\bar{X} - \bar{Y}}{S_{\bar{X} - \bar{Y}}}$


Distribution: $ T$ ~ $t(2N-2) $

---

- Calculate pooled variance

In [116]:
s_p = np.sqrt(( (n_old - 1) * s_old * s_old + (n_new - 1) * s_new * s_new) / (n_old + n_new - 2))
s_p

0.3244894639038153

- Calculate test statistics

In [117]:
T = (mean_old - mean_new) /(s_p * np.sqrt(1 / n_old + 1 / n_new))
T

1.3116069027029211

- Calculate p-value (Single-tail)

<img src="http://www.ttable.org/uploads/2/1/7/9/21795380/published/9754276.png?1517416376" width="400">

In [118]:
from scipy.stats import norm
print(norm.cdf(T))

0.9051735985980927


Conclusion: **CANNOT** reject $H_0$ that old and new has same conversion ratio

# Calculate Sample size / Power


<img src="https://i.ytimg.com/vi/70uNTAP1J-I/maxresdefault.jpg" width="400">

Example: https://www.evanmiller.org/ab-testing/sample-size.html

In [215]:
p_baseline = 0.2
effect_size = 0.05
sig = 0.95 # assume two-tail
sample_size = 1030

- look up table: $Z(\alpha/2) = 1.96$

    
- Generally:
    - $\sigma^2(\bar X - \bar Y) = 2\sigma^2(\bar X) = 2\sigma^2(\frac{x_1 + x_2 + ... + x_n}{N}) = \frac{2\sigma^2_x}{N}$
    - $\bar X - \bar Y$ ~ $N(., \sigma_x \sqrt{2/N})   $
    
      
- for single observation
    - $\sigma^2(x) = p(1-p)$
    - $s^2(x) = \hat p (1-\hat p)$
    
    
- Calculate Standard deviation for average:
    - $\hat p = \bar X$
    - $Var(\hat p) = Var(\bar X) = \frac{p(1-p)}{N}$
    - $s^2(\hat p) = s^2(\bar X) =\frac{\hat p (1-\hat p)}{N-1}$
    - $\bar X$ and $\bar Y$: $Var(\bar X - \bar Y) = \frac{2p(1-p)}{N} $
    - $\bar X$ and $\bar Y$: $s^2(\bar X - \bar Y) = \frac{2\hat p(1-\hat p)}{N} $
    
    
- Calculate power of test
    - Refer to the figure above
    - $ ES_{N(0,1)}  = \frac{ES}{s_x}\sqrt{\frac{N}{2}}$
    - $ ES_{N(0,1)}  = \frac{ES}{s_{\bar X - \bar Y}}$
    - red_shade = $ \phi(Z_{critical} - ES_{N(0,1)})) $
    - power = 1 - red_shade
    
    

In [216]:
s_x = np.sqrt(p_baseline * (1 - p_baseline))
s_x

0.4

In [221]:
s_ES =  s_x * np.sqrt( 2 / sample_size)
s_ES

0.01762610596956927

In [222]:
effect_size_N_0_1 = effect_size / s_ES
effect_size_N_0_1

2.836701429477554

In [223]:
phi_value = 1.96 - effect_size_N_0_1
phi_value

-0.8767014294775541

In [224]:
red_shade = norm.cdf(phi_value)
red_shade

0.19032441506917608

In [220]:
power = 1 - red_shade
power

0.8096755849308239